<a href="https://colab.research.google.com/github/alphoenixbiz/Natural-Language-Processing/blob/master/MBTI%20Personality/MBTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


# Any results you write to the current directory are saved as output.

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train = pd.read_csv('gdrive/My Drive/Colab Notebooks/Kaggle/edsa-mbti/train.csv')
test = pd.read_csv('gdrive/My Drive/Colab Notebooks/Kaggle/edsa-mbti/test.csv')

In [4]:
train.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [5]:
test.head()

,id,posts
0,1,Musical Linguistic Logic & Naturalist (tied)|...
1,2,'You: hello :) Stranger: hii You: how are you ...
2,3,'What worked for me was knowing that limerence...
3,4,'Please forget about him. You should definitel...
4,5,"'Ooh, ENTJ Celestia is actually an interesting..."


# Text Preprocessing

In [0]:
train_type = train[['type']]
test_ID = test['id']
# Now drop the  'Id' colums before the feature engineering process
train.drop(['type'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

In [7]:
# Put all the features to gether for transformation
post_features = pd.concat([train, test],sort=False).reset_index(drop=True)
post_features.head()

,posts
0,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,'I'm finding the lack of me in these posts ver...
2,'Good one _____ https://www.youtube.com/wat...
3,"'Dear INTP, I enjoyed our conversation the o..."
4,'You're fired.|||That's another silly misconce...


In [0]:
# remove a regex pattern 

pattern = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
post_features = post_features.replace(to_replace = pattern, value = '', regex = True)

In [9]:
post_features.head()

,posts
0,'||||||enfp and intj moments sportscenter n...
1,'I'm finding the lack of me in these posts ver...
2,"'Good one _____ |||Of course, to which I sa..."
3,"'Dear INTP, I enjoyed our conversation the o..."
4,'You're fired.|||That's another silly misconce...


In [10]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

lem = WordNetLemmatizer()
# ps = PorterStemmer()
corpus =[]
# cleaning text
for i in range(0,len(post_features['posts'])):

    post = re.sub('[^a-zA-Z]', ' ',str(post_features.iloc[i].values)) 
    post = post.lower()
    post = post.split()
#   ps.stem(word)
    stop_wrds = set(stopwords.words('english'))
    post = [lem.lemmatize(word, "v") for word in post if not word in stop_wrds]
    post = ' '.join(post)
    corpus.append(post)

post_features['posts_clean'] = corpus
post_features.drop(['posts'], axis=1, inplace=True)              

In [12]:
post_features.head()

,posts_clean
0,enfp intj moments sportscenter top ten play pr...
1,find lack post alarm sex bore position often e...
2,good one course say know bless curse absolutel...
3,dear intp enjoy conversation day esoteric gabb...
4,fire another silly misconception approach logi...


# Text to Features 

In [13]:
# Statistical Features
obj = TfidfVectorizer()
post_final_features = obj.fit_transform(post_features['posts_clean'])
print(post_final_features[0])

# obj = CountVectorizer()
# post_final_features = obj.fit_transform(post_features['posts_clean']).toarray()
# print(post_final_features[0:5])

  (0, 51321)	0.022971753095914153
  (0, 71096)	0.01986360970350271
  (0, 4062)	0.047274026208539495
  (0, 18316)	0.09498645008638615
  (0, 82020)	0.01596645415910647
  (0, 14567)	0.053867316511501555
  (0, 47093)	0.06500552884892956
  (0, 54808)	0.07320038515485536
  (0, 25468)	0.06060195869391936
  (0, 7586)	0.027894891259623238
  (0, 58435)	0.04538429404512145
  (0, 7408)	0.021206654335971402
  (0, 69879)	0.029399599740099487
  (0, 1498)	0.027917308013944236
  (0, 84642)	0.02183200436723868
  (0, 70189)	0.028175502555248942
  (0, 60135)	0.043232650308424714
  (0, 25455)	0.041018094693615934
  (0, 1400)	0.032429904556302395
  (0, 49908)	0.02854634446563593
  (0, 65751)	0.024802541253978988
  (0, 53456)	0.027598014612381074
  (0, 1600)	0.06274183478703248
  (0, 84762)	0.04596298373355101
  (0, 71609)	0.06592524469693782
  :	:
  (0, 51400)	0.03262506371186185
  (0, 72390)	0.059059136897429226
  (0, 14036)	0.052354713799169145
  (0, 25335)	0.04572854782274967
  (0, 58495)	0.0187643849744

In [14]:
# create our target variables
pd.options.mode.chained_assignment = None

train_type['I-E'] = train_type['type'].astype(str).str[0]
train_type['I-E'] = train_type['I-E'].map({"I": 0, "E": 1})
train_type['S-N'] = train_type['type'].astype(str).str[1]
train_type['S-N'] = train_type['S-N'].map({"S": 0, "N": 1})
train_type['F-T'] = train_type['type'].astype(str).str[2]
train_type['F-T'] = train_type['F-T'].map({"F": 0, "T": 1})
train_type['P-J'] = train_type['type'].astype(str).str[3]
train_type['P-J'] = train_type['P-J'].map({"P": 0, "J": 1})
train_type.drop('type', axis=1, inplace=True) 
y = train_type
print(y[0:5])

   I-E  S-N  F-T  P-J
0    0    1    0    1
1    1    1    1    0
2    0    1    1    0
3    0    1    1    1
4    1    1    1    1


In [15]:

y = np.array(y)
print(y[0:5])

[[0 1 0 1]
 [1 1 1 0]
 [0 1 1 0]
 [0 1 1 1]
 [1 1 1 1]]


In [16]:
# Spliting the data back to train(X,y) and test(X_sub)
X = post_final_features[:len(y), :]
X_test = post_final_features[len(y):, :]
print('Features size for train(X,y) and test(X_test):')
print('X', X.shape, 'y', y.shape, 'X_test', X_test.shape)

Features size for train(X,y) and test(X_test):
X (6506, 85722) y (6506, 4) X_test (2169, 85722)


# Text Classification

In [0]:
# from sklearn.naive_bayes import MultinomialNB
# model = OneVsRestClassifier(MultinomialNB())

In [0]:
model = OneVsRestClassifier(LogisticRegressionCV(Cs=30,l1_ratios=[1],penalty='elasticnet', solver = 'saga',multi_class = 'multinomial', cv=10), n_jobs =-1)

In [0]:
model.fit(X, y)

In [0]:
y_predicted = np.round(np.floor(model.predict_proba(X_test)),2)

In [0]:
y_predicted[0:5]

# Submission

In [0]:
submission = pd.DataFrame({'id' : np.array(test_ID),'mind' : y_predicted[:,0], 
                           'energy' : y_predicted[:,1], 'nature' : y_predicted[:,2], 
                           'tactics' : y_predicted[:,3]})
print('Save submission')

In [0]:

submission.to_csv("gdrive/My Drive/Colab Notebooks/Kaggle/new_submission.csv", index=False)
submission.head()